## Aprendizaje de reglas de asociación

Objetivo:
    
    derivar reglas de la forma {A} -> {B}
    

In [1]:
import numpy as np
from itertools import combinations, groupby
from collections import Counter

import pandas as pd
import sys
from IPython.display import display
from efficient_apriori import apriori

In [2]:
#Obtenemos el Id de las peliculas con sus nombres asociados
name_pelis = pd.read_csv('ml-20m/movies.csv')
display(name_pelis.shape, name_pelis.head(10))

(27278, 3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [3]:
#Obtenemos el dataset que contiene la info de que/cuantos/cuales usuarios vieron que/cuantas/cuales películas.
ratings = pd.read_csv('ml-20m/ratings.csv')
#ratings.head(10)

In [4]:
#Funcion que nos da el tamaño del objeto
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))

In [5]:
print('ratings -- dimensions: {0};   size: {1}'.format(ratings.shape, size(ratings))) #OOOhhh :'/

ratings -- dimensions: (20000263, 4);   size: 640.01 MB


In [6]:
#Armamos un nuevo Dataset que contenga la info necesaria de los dos cvs anteriores (Usuario -> peliculas vistas (y nombre de las pelis))
vistas_df = pd.merge(ratings[['userId','movieId']], name_pelis[['movieId','title']] ,on='movieId', how= "inner")
display(vistas_df.shape, vistas_df.head(10))

(20000263, 3)

,userId,movieId,title
0,1,2,Jumanji (1995)
1,5,2,Jumanji (1995)
2,13,2,Jumanji (1995)
3,29,2,Jumanji (1995)
4,34,2,Jumanji (1995)
5,54,2,Jumanji (1995)
6,88,2,Jumanji (1995)
7,91,2,Jumanji (1995)
8,116,2,Jumanji (1995)
9,119,2,Jumanji (1995)


In [7]:
#Ordenamos el Dataset por Clientes
vistas_df=vistas_df.sort_values( by='userId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
display(vistas_df.shape,vistas_df.head(10))

(20000263, 3)

,userId,movieId,title
0,1,2,Jumanji (1995)
505014,1,541,Blade Runner (1982)
2380423,1,6807,Monty Python's The Meaning of Life (1983)
2378699,1,6774,Videodrome (1983)
2376750,1,6755,Bubba Ho-tep (2002)
2372550,1,6754,Underworld (2003)
535540,1,589,Terminator 2: Judgment Day (1991)
1837151,1,2918,Ferris Bueller's Day Off (1986)
1536213,1,2100,Splash (1984)
1860468,1,2944,"Dirty Dozen, The (1967)"


In [8]:
#Pasamos la info necesaria (userId,movieId) a un array,para así poder implementar el algoritmo "Apriori"
vistas = vistas_df.values[:,[0,2]]
display(vistas.shape,vistas)

(20000263, 2)

array([[1, 'Jumanji (1995)'],
       [1, 'Blade Runner (1982)'],
       [1, "Monty Python's The Meaning of Life (1983)"],
       ...,
       [138493, 'Tango & Cash (1989)'],
       [138493, 'Chain Reaction (1996)'],
       [138493, 'Prestige, The (2006)']], dtype=object)

In [9]:
transactions=[]
for userId, peli_vista in groupby(vistas, lambda x: x[0]):
    transactions.append([item[1] for item in peli_vista])
print(transactions)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
#Como el Dataset es muy grande para correr las transacciones, entonces, seleccionamos un doscientosavo(?) del original

vistas = vistas_df.values[:(round(vistas.shape[0]/200)),[0,2]]
display(vistas.shape,vistas)

(100001, 2)

array([[1, 'Jumanji (1995)'],
       [1, 'Blade Runner (1982)'],
       [1, "Monty Python's The Meaning of Life (1983)"],
       ...,
       [702, 'Vertigo (1958)'],
       [702, 'Couch Trip, The (1988)'],
       [702, 'Birds, The (1963)']], dtype=object)

In [11]:
transactions=[]
for userId, peli_vista in groupby(vistas, lambda x: x[0]):
    transactions.append([item[1] for item in peli_vista])
print(transactions)

[['Jumanji (1995)', 'Blade Runner (1982)', "Monty Python's The Meaning of Life (1983)", 'Videodrome (1983)', 'Bubba Ho-tep (2002)', 'Underworld (2003)', 'Terminator 2: Judgment Day (1991)', "Ferris Bueller's Day Off (1986)", 'Splash (1984)', 'Dirty Dozen, The (1967)', 'Seven (a.k.a. Se7en) (1995)', 'Young Frankenstein (1974)', 'Goldfinger (1964)', 'Time Machine, The (2002)', 'Monty Python and the Holy Grail (1975)', 'Full Metal Jacket (1987)', 'Shrek (2001)', 'Vampire Hunter D: Bloodlust (Banpaia hantâ D) (2000)', 'Dead Zone, The (1983)', 'Dead Poets Society (1989)', 'Christmas Story, A (1983)', 'Lord of the Rings: The Return of the King, The (2003)', "One Flew Over the Cuckoo's Nest (1975)", 'Brotherhood of the Wolf (Pacte des loups, Le) (2001)', 'Witches of Eastwick, The (1987)', 'Watership Down (1978)', 'Dragonslayer (1981)', 'Witches, The (1990)', 'Invasion of the Body Snatchers (1978)', 'Conan the Destroyer (1984)', 'Scary Movie 3 (2003)', 'Wilder Napalm (1993)', 'Indiana Jones an

In [12]:
itemsets, rules = apriori(transactions, min_support=0.31, min_confidence=0.1)

In [13]:
rules, itemsets

([{Jurassic Park (1993)} -> {Forrest Gump (1994)},
  {Forrest Gump (1994)} -> {Jurassic Park (1993)},
  {Pulp Fiction (1994)} -> {Forrest Gump (1994)},
  {Forrest Gump (1994)} -> {Pulp Fiction (1994)},
  {Shawshank Redemption, The (1994)} -> {Pulp Fiction (1994)},
  {Pulp Fiction (1994)} -> {Shawshank Redemption, The (1994)},
  {Silence of the Lambs, The (1991)} -> {Pulp Fiction (1994)},
  {Pulp Fiction (1994)} -> {Silence of the Lambs, The (1991)}],
 {1: {('American Beauty (1999)',): 238,
   ('Apollo 13 (1995)',): 234,
   ('Batman (1989)',): 219,
   ('Braveheart (1995)',): 261,
   ('Fargo (1996)',): 229,
   ('Forrest Gump (1994)',): 339,
   ('Fugitive, The (1993)',): 233,
   ('Independence Day (a.k.a. ID4) (1996)',): 234,
   ('Jurassic Park (1993)',): 301,
   ('Matrix, The (1999)',): 253,
   ('Pulp Fiction (1994)',): 350,
   ("Schindler's List (1993)",): 246,
   ('Shawshank Redemption, The (1994)',): 304,
   ('Silence of the Lambs, The (1991)',): 294,
   ('Star Wars: Episode IV - A Ne

In [14]:
rules=sorted(rules, key=lambda rule: rule.confidence)
cont = 0
for rule in rules:
    print(rule) # Prints the rule and its confidence, support, lift, ...
    cont+=1
    if cont > 100:
        break

{Pulp Fiction (1994)} -> {Shawshank Redemption, The (1994)} (conf: 0.629, supp: 0.313, lift: 1.452, conv: 1.526)
{Pulp Fiction (1994)} -> {Forrest Gump (1994)} (conf: 0.640, supp: 0.319, lift: 1.325, conv: 1.436)
{Pulp Fiction (1994)} -> {Silence of the Lambs, The (1991)} (conf: 0.649, supp: 0.323, lift: 1.549, conv: 1.654)
{Forrest Gump (1994)} -> {Pulp Fiction (1994)} (conf: 0.661, supp: 0.319, lift: 1.325, conv: 1.478)
{Forrest Gump (1994)} -> {Jurassic Park (1993)} (conf: 0.714, supp: 0.345, lift: 1.665, conv: 1.996)
{Shawshank Redemption, The (1994)} -> {Pulp Fiction (1994)} (conf: 0.724, supp: 0.313, lift: 1.452, conv: 1.815)
{Silence of the Lambs, The (1991)} -> {Pulp Fiction (1994)} (conf: 0.772, supp: 0.323, lift: 1.549, conv: 2.200)
{Jurassic Park (1993)} -> {Forrest Gump (1994)} (conf: 0.804, supp: 0.345, lift: 1.665, conv: 2.638)


In [15]:
itemsets, rules = apriori(transactions, min_support=0.1, min_confidence=0.1)
rules, itemsets

([{Alien (1979)} -> {2001: A Space Odyssey (1968)},
  {2001: A Space Odyssey (1968)} -> {Alien (1979)},
  {Aliens (1986)} -> {2001: A Space Odyssey (1968)},
  {2001: A Space Odyssey (1968)} -> {Aliens (1986)},
  {Apocalypse Now (1979)} -> {2001: A Space Odyssey (1968)},
  {2001: A Space Odyssey (1968)} -> {Apocalypse Now (1979)},
  {Back to the Future (1985)} -> {2001: A Space Odyssey (1968)},
  {2001: A Space Odyssey (1968)} -> {Back to the Future (1985)},
  {Blade Runner (1982)} -> {2001: A Space Odyssey (1968)},
  {2001: A Space Odyssey (1968)} -> {Blade Runner (1982)},
  {Clockwork Orange, A (1971)} -> {2001: A Space Odyssey (1968)},
  {2001: A Space Odyssey (1968)} -> {Clockwork Orange, A (1971)},
  {E.T. the Extra-Terrestrial (1982)} -> {2001: A Space Odyssey (1968)},
  {2001: A Space Odyssey (1968)} -> {E.T. the Extra-Terrestrial (1982)},
  {Fargo (1996)} -> {2001: A Space Odyssey (1968)},
  {2001: A Space Odyssey (1968)} -> {Fargo (1996)},
  {Forrest Gump (1994)} -> {2001: A Sp

In [16]:
rules=sorted(rules, key=lambda rule: rule.confidence)
cont = 0
for rule in rules:
    print(rule) # Prints the rule and its confidence, support, lift, ...
    cont+=1
    if cont > 12:
        break

{Pulp Fiction (1994)} -> {Casablanca (1942)} (conf: 0.203, supp: 0.101, lift: 1.167, conv: 1.036)
{Pulp Fiction (1994)} -> {Demolition Man (1993)} (conf: 0.203, supp: 0.101, lift: 1.453, conv: 1.079)
{Pulp Fiction (1994)} -> {Ferris Bueller's Day Off (1986)} (conf: 0.203, supp: 0.101, lift: 1.369, conv: 1.069)
{Pulp Fiction (1994)} -> {Finding Nemo (2003)} (conf: 0.203, supp: 0.101, lift: 1.331, conv: 1.063)
{Pulp Fiction (1994)} -> {Sling Blade (1996)} (conf: 0.203, supp: 0.101, lift: 1.675, conv: 1.103)
{Pulp Fiction (1994)} -> {Top Gun (1986)} (conf: 0.203, supp: 0.101, lift: 1.410, conv: 1.074)
{Pulp Fiction (1994)} -> {Aladdin (1992), Interview with the Vampire: The Vampire Chronicles (1994)} (conf: 0.203, supp: 0.101, lift: 1.618, conv: 1.097)
{Pulp Fiction (1994)} -> {Alien (1979), American Beauty (1999)} (conf: 0.203, supp: 0.101, lift: 1.758, conv: 1.110)
{Pulp Fiction (1994)} -> {Alien (1979), Terminator 2: Judgment Day (1991)} (conf: 0.203, supp: 0.101, lift: 1.383, conv: 1.

In [17]:
rules=sorted(rules, key=lambda rule: rule.support)
cont = 0
for rule in rules:
    print(rule) # Prints the rule and its confidence, support, lift, ...
    cont+=1
    if cont > 12:
        break

{Pulp Fiction (1994)} -> {Casablanca (1942)} (conf: 0.203, supp: 0.101, lift: 1.167, conv: 1.036)
{Pulp Fiction (1994)} -> {Demolition Man (1993)} (conf: 0.203, supp: 0.101, lift: 1.453, conv: 1.079)
{Pulp Fiction (1994)} -> {Ferris Bueller's Day Off (1986)} (conf: 0.203, supp: 0.101, lift: 1.369, conv: 1.069)
{Pulp Fiction (1994)} -> {Finding Nemo (2003)} (conf: 0.203, supp: 0.101, lift: 1.331, conv: 1.063)
{Pulp Fiction (1994)} -> {Sling Blade (1996)} (conf: 0.203, supp: 0.101, lift: 1.675, conv: 1.103)
{Pulp Fiction (1994)} -> {Top Gun (1986)} (conf: 0.203, supp: 0.101, lift: 1.410, conv: 1.074)
{Pulp Fiction (1994)} -> {Aladdin (1992), Interview with the Vampire: The Vampire Chronicles (1994)} (conf: 0.203, supp: 0.101, lift: 1.618, conv: 1.097)
{Pulp Fiction (1994)} -> {Alien (1979), American Beauty (1999)} (conf: 0.203, supp: 0.101, lift: 1.758, conv: 1.110)
{Pulp Fiction (1994)} -> {Alien (1979), Terminator 2: Judgment Day (1991)} (conf: 0.203, supp: 0.101, lift: 1.383, conv: 1.

In [18]:
rules=sorted(rules, key=lambda rule: rule.conviction)
cont = 0
for rule in rules:
    print(rule) # Prints the rule and its confidence, support, lift, ...
    cont+=1
    if cont > 12:
        break

{Apollo 13 (1995)} -> {American Beauty (1999)} (conf: 0.329, supp: 0.110, lift: 0.971, conv: 0.985)
{American Beauty (1999)} -> {Apollo 13 (1995)} (conf: 0.324, supp: 0.110, lift: 0.971, conv: 0.986)
{True Lies (1994)} -> {American Beauty (1999)} (conf: 0.330, supp: 0.105, lift: 0.974, conv: 0.987)
{American Beauty (1999)} -> {True Lies (1994)} (conf: 0.311, supp: 0.105, lift: 0.974, conv: 0.988)
{Dances with Wolves (1990)} -> {American Beauty (1999)} (conf: 0.335, supp: 0.103, lift: 0.988, conv: 0.994)
{American Beauty (1999)} -> {Dances with Wolves (1990)} (conf: 0.303, supp: 0.103, lift: 0.988, conv: 0.995)
{Dances with Wolves (1990)} -> {Matrix, The (1999)} (conf: 0.358, supp: 0.110, lift: 0.994, conv: 0.996)
{Matrix, The (1999)} -> {Dances with Wolves (1990)} (conf: 0.304, supp: 0.110, lift: 0.994, conv: 0.997)
{Batman (1989)} -> {American Beauty (1999)} (conf: 0.338, supp: 0.105, lift: 0.997, conv: 0.998)
{American Beauty (1999)} -> {Batman (1989)} (conf: 0.311, supp: 0.105, lift

In [19]:
rules=sorted(rules, key=lambda rule: rule.lift)
cont = 0
for rule in rules:
    print(rule) # Prints the rule and its confidence, support, lift, ...
    cont+=1
    if cont > 12:
        break

{Apollo 13 (1995)} -> {American Beauty (1999)} (conf: 0.329, supp: 0.110, lift: 0.971, conv: 0.985)
{American Beauty (1999)} -> {Apollo 13 (1995)} (conf: 0.324, supp: 0.110, lift: 0.971, conv: 0.986)
{True Lies (1994)} -> {American Beauty (1999)} (conf: 0.330, supp: 0.105, lift: 0.974, conv: 0.987)
{American Beauty (1999)} -> {True Lies (1994)} (conf: 0.311, supp: 0.105, lift: 0.974, conv: 0.988)
{Dances with Wolves (1990)} -> {American Beauty (1999)} (conf: 0.335, supp: 0.103, lift: 0.988, conv: 0.994)
{American Beauty (1999)} -> {Dances with Wolves (1990)} (conf: 0.303, supp: 0.103, lift: 0.988, conv: 0.995)
{Dances with Wolves (1990)} -> {Matrix, The (1999)} (conf: 0.358, supp: 0.110, lift: 0.994, conv: 0.996)
{Matrix, The (1999)} -> {Dances with Wolves (1990)} (conf: 0.304, supp: 0.110, lift: 0.994, conv: 0.997)
{Batman (1989)} -> {American Beauty (1999)} (conf: 0.338, supp: 0.105, lift: 0.997, conv: 0.998)
{American Beauty (1999)} -> {Batman (1989)} (conf: 0.311, supp: 0.105, lift

### Conclusiones

###### Primero y principal
Entiendo que en este caso los resultados obtenidos son muy poco representativos, porque el número de muestras que utilizamos fue del 0.5% (1/200), por lo que tenemos un numero muy reducido de usuarios que vieron peliculas (transacciones) en comparacion con el Dataset original, ya que el orden exponencial de las mismas nos impedía que la máquina pudiera realizar los cálculos (armar el array de transacciones).


###### Segundo y secundario   
En un principio nos enfocamos en un min_support "lo más alto posible" (31), que entendemos como el treshold o umbral mínimo de frecuencia con la que aparece un ítem, en este caso, la peli. (P(peli)). Que es el que nos da el punto de corte, ya que la "confidence" tiene un margen de valores mucho más altos.

Este primer filtro nos devolvió una cantidad muy reducida de peliculas y asociaciones, pero con la característica de tener valores "muy elevados" de confianza.

Teniendo en cuenta que la confianza es la probabilidad condicional de los items. P(Peli1|peli0)
Esta nos da una "fortaleza" de que dado que ocurrió un evento, ocurrirá otro.

En el primer ejemplo que realizamos observamos que hay una confianza muy fuerte (80%) en que quien vió "Jurassic Park" entonces verá "Forrest Gump". 
El problema que entiendo se nos puede presentar con esta (u otra) asociación es que por si sola puede ser irrelevante, ya que si el valor de lift es cercano a 1, entonces la asociación sería "puramente casual", y el 80% del confidence carecería de valor.
Desconozco cuales son los umbrales críticos del lift para determinar si los elementos son posibles complementos (lift > 1) o si son posibles sustitutos (lift < 1).

Realizando un segundo ejemplo con un valor de min_support reducido, obtenemos mayor cantidad de asociaciones, y la diferencia entre las reglas es más notoria, pero también notamos que las confianzas disminuyeron y que los lift soy en su mayoría mucho más cercanos a 1, con lo que ('hablado' en el parrafo anterior) eso implica.